# Question 6

In [1]:
import json
import datetime,time
import pytz
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statistics import mean
import time

In [2]:
path = "../../projects/project_5/ECE219_tweet_data/"

files = ["tweets_#gohawks.txt", "tweets_#gopatriots.txt", \
        "tweets_#nfl.txt", "tweets_#patriots.txt", \
        "tweets_#sb49.txt", "tweets_#superbowl.txt"]
topics = ["gohawks", "gopatriots", "nfl", "patriots", "sb49", "superbowl"]

for i, fl in enumerate(files):
    print("files[" + str(i) + "] => " + fl)

files[0] => tweets_#gohawks.txt
files[1] => tweets_#gopatriots.txt
files[2] => tweets_#nfl.txt
files[3] => tweets_#patriots.txt
files[4] => tweets_#sb49.txt
files[5] => tweets_#superbowl.txt


#### Prepare for data

In [3]:
def transfer_time(data_raw,time_type):
    
    pst_tz = pytz.timezone('America/Los_Angeles')
    
    # sort according to time
    pddata_raw = pd.DataFrame(data_raw,columns=['time','tweets','retweets','followers','mentioned',\
                                                'media','active','author','favourites_count','title'])
    pddata_raw = pddata_raw.sort_values(by = 'time')
    pddata_raw = pddata_raw.reset_index(drop=True)               

    # calculate hour index and minute index from time
    if time_type == 'hour':
        hour_accu = []
        for index, row in pddata_raw.iterrows():  
            p = datetime.datetime.fromtimestamp(row['time'], pst_tz)  
            hour_accu.append(((p.month-1)*31+p.day-14)*24+p.hour)                             
        pddata_raw['time'] = hour_accu
    elif time_type == 'minute':
        minu_accu = []
        for index, row in pddata_raw.iterrows():  
            p = datetime.datetime.fromtimestamp(row['time'], pst_tz)                    
            minu_accu.append((((p.month-1)*31+p.day-14)*24 + (p.hour-0))*12 + p.minute//5)             
        pddata_raw['time'] = minu_accu    
    else:
        print("Invalid time type")
        
    return pddata_raw

In [4]:
def generate_df(pddata_raw):
    """
    Create a new dataframe with desired form
    """
    df = pd.DataFrame([],columns=['time unit','tweets','retweets','followers sum','followers max',\
                                  'mentioned','media','active','author','favourites_count','title'])
    
    col = pddata_raw.columns.get_loc('time')
    df['time unit'] = range(int(pddata_raw.iloc[len(pddata_raw.index)-1,col] - pddata_raw.iloc[0,col]+1))

    df['tweets'] = pddata_raw.groupby("time")['tweets'].sum()
    df['retweets'] = pddata_raw.groupby("time")['retweets'].sum()
    df['followers sum'] = pddata_raw.groupby("time")['followers'].sum()
    df['followers max'] = pddata_raw.groupby("time")["followers"].max()
    df['mentioned'] = pddata_raw.groupby("time")['mentioned'].sum()
    df['media'] = pddata_raw.groupby("time")['media'].sum()
    df['active'] = pddata_raw.groupby("time")['active'].mean()  
    df['author'] = pddata_raw.groupby("time")['author'].nunique() # count number of not-repeating authors    
    df['favourites_count'] = pddata_raw.groupby("time")['favourites_count'].sum()
    df['title'] = pddata_raw.groupby("time")['title'].mean()
            
    # reset index of df
    df = df.fillna(0).reset_index(drop=True)
    
    return df

In [5]:
def prase_dataset(file):
    """
    Prase x conponent of the dataset into pandas DataFrame including columns of:
    tweets, retweets, total_followers, max_followers, mentioned, media, active, author, favourites_count, title
    with lines of hours,
    where mentioned: number of @ in tweets per hour
          media: number of url in tweets per hour
          active: a measure of active state of author
          author: number of unique authors post tweet per hour
          favourites_count: the total number of likes by this user
          title: length of this tweet's title
    Prase y of dataset as number of tweets in the next hour.
    """
    start_time = time.mktime(time.strptime("2015-02-01 08:00:00",'%Y-%m-%d %H:%M:%S'))
    end_time = time.mktime(time.strptime("2015-02-01 20:00:00",'%Y-%m-%d %H:%M:%S'))   

    start_hour_idx = ((2-1)*31+1-14)*24+8
    end_hour_idx = ((2-1)*31+1-14)*24+20
    start_minute_idx = (((2-1)*31+1-14)*24 + (8-0))*12 + 0//5    
        
    # extract raw features
    data_raw = [[],[],[]]
    for line in open(path + file, 'r') :
        row_tmp = []
        a = json.loads(line)
        citation_date = a['citation_date']
        tweet = 1
        retweet = a['metrics']['citations']['total']
        foll = a['author']['followers']             
        ment = len(a['tweet']['entities']['user_mentions'])        
        medi = len(a['tweet']['extended_entities']['media']) if 'extended_entities' in a['tweet'] else 0
        hist_tw = a['tweet']['user']["statuses_count"]
        hist_yr = a['tweet']['user']['created_at'][-4:]
        acti = hist_tw/(2015-float(hist_yr)+1) 
        auth = a['author']['name']
        favo = a['tweet']['user']['favourites_count']
        titl = len(a['title'])
        
        # append to row_tmp
        row_tmp.append(citation_date)        
        row_tmp.append(tweet)        
        row_tmp.append(retweet)
        row_tmp.append(foll)    
        row_tmp.append(ment) 
        row_tmp.append(medi) 
        row_tmp.append(acti)  
        row_tmp.append(auth)
        row_tmp.append(favo)
        row_tmp.append(titl)
        
        # assign to 3 periods
        if citation_date < start_time:
            data_raw[0].append(row_tmp)
        elif citation_date < end_time:
            data_raw[1].append(row_tmp)
        else:
            data_raw[2].append(row_tmp)            

    # generate raw pandas dataframe
    pddata_raw_1 = transfer_time(data_raw[0],'hour')
    pddata_raw_2 = transfer_time(data_raw[1],'minute')
    pddata_raw_2['time'] = pddata_raw_2['time'] - start_minute_idx
    pddata_raw_3 = transfer_time(data_raw[2],'hour')
    pddata_raw_3['time'] = pddata_raw_3['time'] - end_hour_idx - 1    
            
    # generate df and df_y for each time slot
    df_1 = generate_df(pddata_raw_1)  
    df_y_1 = df_1.iloc[1:,1].reset_index(drop=True)
    df_1 = df_1[:len(df_y_1)]
    
    df_2 = generate_df(pddata_raw_2)
    df_y_2 = df_2.iloc[1:,1].reset_index(drop=True)
    df_2 = df_2[:len(df_y_2)]
   
    df_3 = generate_df(pddata_raw_3)
    df_y_3 = df_3.iloc[1:,1].reset_index(drop=True)
    df_3 = df_3[:len(df_y_3)]
    
    return (df_1.iloc[:,1:],df_y_1), (df_2.iloc[:,1:],df_y_2), (df_3.iloc[:,1:],df_y_3)

#### Train linear regression model

In [6]:
def train_lr(i):
    """
    This function gets data and train using a linear regression model
    """    
    (df_1,df_y_1),(df_2,df_y_2),(df_3,df_y_3) = prase_dataset(files[i])
    print('Parse data finished','\n')

    reg_1 = LinearRegression().fit(df_1, df_y_1)
    pred_y_1 = reg_1.predict(df_1)
    MSE_1 = mean_squared_error(df_y_1, pred_y_1)
    R2_1 = r2_score(df_y_1, pred_y_1)    
    print(topics[i], ' of time period 1')
    print('MSE for test data = ',MSE_1)
    print('R2 score for test data = ',R2_1,'\n')   

    reg_2 = LinearRegression().fit(df_2, df_y_2)
    pred_y_2 = reg_2.predict(df_2)
    MSE_2 = mean_squared_error(df_y_2, pred_y_2)
    R2_2 = r2_score(df_y_2, pred_y_2)    
    print(topics[i], ' of time period 2')
    print('MSE for test data = ',MSE_2)
    print('R2 score for test data = ',R2_2,'\n') 

    reg_3 = LinearRegression().fit(df_3, df_y_3)
    pred_y_3 = reg_3.predict(df_3)
    MSE_3 = mean_squared_error(df_y_3, pred_y_3)
    R2_3 = r2_score(df_y_3, pred_y_3)    
    print(topics[i], ' of time period 3')
    print('MSE for test data = ',MSE_3)
    print('R2 score for test data = ',R2_3,'\n') 
    
    return (df_1, df_y_1, pred_y_1), (df_2, df_y_2, pred_y_2), (df_3, df_y_3, pred_y_3)

## Results

#### gohawks

In [7]:
(df_1, df_y_1, pred_y_1), (df_2, df_y_2, pred_y_2), (df_3, df_y_3, pred_y_3) = train_lr(0)

Parse data finished 

gohawks  of time period 1
MSE for test data =  414263.04817109066
R2 score for test data =  0.598161945585284 

gohawks  of time period 2
MSE for test data =  63160.147810741204
R2 score for test data =  0.5547907515717387 

gohawks  of time period 3
MSE for test data =  919.8478574822856
R2 score for test data =  0.8959158441687535 



#### gopatriots

In [8]:
(df_1, df_y_1, pred_y_1), (df_2, df_y_2, pred_y_2), (df_3, df_y_3, pred_y_3) = train_lr(1)

Parse data finished 

gopatriots  of time period 1
MSE for test data =  1196.4484741182887
R2 score for test data =  0.7080866576767786 

gopatriots  of time period 2
MSE for test data =  13902.440230090087
R2 score for test data =  0.45870700231074846 

gopatriots  of time period 3
MSE for test data =  2.734346947494579
R2 score for test data =  0.981672264517095 



#### nfl

In [9]:
(df_1, df_y_1, pred_y_1), (df_2, df_y_2, pred_y_2), (df_3, df_y_3, pred_y_3) = train_lr(2)

Parse data finished 

nfl  of time period 1
MSE for test data =  59097.839883621986
R2 score for test data =  0.5622301246225405 

nfl  of time period 2
MSE for test data =  15448.2604714677
R2 score for test data =  0.8662712504630752 

nfl  of time period 3
MSE for test data =  15726.387836028154
R2 score for test data =  0.8193018818392885 



#### patriots

In [10]:
(df_1, df_y_1, pred_y_1), (df_2, df_y_2, pred_y_2), (df_3, df_y_3, pred_y_3) = train_lr(3)

Parse data finished 

patriots  of time period 1
MSE for test data =  292559.76189079345
R2 score for test data =  0.6255415231220794 

patriots  of time period 2
MSE for test data =  592589.0646312773
R2 score for test data =  0.7443493708746511 

patriots  of time period 3
MSE for test data =  5485.772795474432
R2 score for test data =  0.9307790836817433 



#### sb49

In [11]:
(df_1, df_y_1, pred_y_1), (df_2, df_y_2, pred_y_2), (df_3, df_y_3, pred_y_3) = train_lr(4)

Parse data finished 

sb49  of time period 1
MSE for test data =  3967.9293381128236
R2 score for test data =  0.8726677185837142 

sb49  of time period 2
MSE for test data =  1061154.1755915731
R2 score for test data =  0.889568015315917 

sb49  of time period 3
MSE for test data =  16093.080413140482
R2 score for test data =  0.951778592832415 



#### superbowl

In [12]:
(df_1, df_y_1, pred_y_1), (df_2, df_y_2, pred_y_2), (df_3, df_y_3, pred_y_3) = train_lr(5)

Parse data finished 

superbowl  of time period 1
MSE for test data =  460135.94631149515
R2 score for test data =  0.4667714326573307 

superbowl  of time period 2
MSE for test data =  4188144.4709006567
R2 score for test data =  0.9329302244854544 

superbowl  of time period 3
MSE for test data =  86282.69151023673
R2 score for test data =  0.8777853512797381 

